# 7_一歩進んだ予測モデルの構築
- 前章では「性別」という単一のデータから生死を予測する単回帰モデルを構築しました
- しかし、性別以外にも生死予測の参考になるデータはあるはずで、それらも考慮した予測モデルの方が精度が高くなると考えられます
- 今回は複数の特徴量から生死を予測する重回帰モデルの構築を行います

## ■写経

### ライブラリインポート（PandasとSciPy）
- 表形式データを簡単に扱うためのPandasをプログラムにインポートします。インポートすることでPandasの機能が有効化されます
- また予測モデルの構築のためにsklearnをインポートします

In [4]:
# 欠損値補完用のクラス
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer

### データ読み込み（前章と同じ）
- Pandasの機能を使い、"train.csv"というデータをPythonコードで読み込み、Dataframeオブジェクトとして扱います

In [5]:
df_train = pd.read_csv("train.csv")

### データを見てみる（前章と同じ）
- head関数をつかって、先頭5行分のデータをダイジェスト表示します

In [7]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### 前準備
- maleを0、femaleを1に置き換えた列を作成します

In [8]:
df_train["Sex_bi"] = df_train["Sex"].map({"male":0, "female":1})

In [9]:
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_bi
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,1
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,0


### 重回帰モデルで使用する特徴量と目的変数を定義し、欠損値処理をする
- 今回は性別、年齢、運賃（特徴量）が生死（目的変数）と関係するという仮説をたてています
- 今回はAgeに欠損値がある可能性があり、それを平均値で補完します

In [16]:
# 特徴量と目的変数定義
features = ["Sex_bi", "Age", "Fare"]
target = "Survived"

# 欠損値処理
# simpleimputerを使うと、欠損値を平均値で埋めることができる(skcit-learnのimputeモジュール)
imputer = SimpleImputer()
df_train[features] = imputer.fit_transform(df_train[features])

# 特徴量と目的変数のデータを分割
x_train = df_train [features]
y_train = df_train [target]

In [19]:
_train

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

### 重回帰モデルの作成と学習

In [6]:
# 空の予測モデルを作成
model = LinearRegression()

# 学習データを与えて学習
model.fit(x_train, y_train)

In [21]:
x_train

,Sex_bi,Age,Fare
0,0.0,22.000000,7.2500
1,1.0,38.000000,71.2833
2,1.0,26.000000,7.9250
3,1.0,35.000000,53.1000
4,0.0,35.000000,8.0500
...,...,...,...
886,0.0,27.000000,13.0000
887,1.0,19.000000,30.0000
888,1.0,29.699118,23.4500
889,0.0,26.000000,30.0000


In [22]:
y_train

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [20]:
model = LinearRegression()
model.fit(x_train, y_train)

LinearRegression()

In [23]:
model.coef_
# y = a1*x1 + a2*x2 + a3*x3 + b

array([ 0.51824834, -0.00158618,  0.0016486 ])

In [24]:
model.intercept_
# bに該当

0.19521710018554136

### テストデータに対する予測の実施
- 上記で学習したモデルに、テスト用のデータを予測させます

In [39]:
# 提出用テストデータを読み込みます（すなわち、学習したモデルの精度を測るための検証データセットです）
df_test = pd.read_csv("test.csv")
# 提出用データも同様に、'Sex'列を0、1に変換します
df_test["Sex_bi"] = df_test["Sex"].map({"male":0,"female":1})
# 提出用データの'Age'に欠損値がある場合も平均値で埋めます
df_test[features] = imputer.transform(df_test[features])
# 提出用データの特徴量を分割します
x_test = df_test [features]
# 提出用データに対する予測を行います
predictions = model.predict(x_test)
predictions

array([0.15340112, 0.65045518, 0.11284475, 0.16667126, 0.69882669,
       0.18821894, 0.67845755, 0.20178592, 0.69683228, 0.20172109,
       0.16112598, 0.1651165 , 0.8126085 , 0.13815143, 0.7397683 ,
       0.72109773, 0.16006104, 0.17381847, 0.68370375, 0.65399848,
       0.20590424, 0.18616887, 0.71859049, 0.2630973 , 1.06988124,
       0.13981283, 0.78074862, 0.1714392 , 0.18046612, 0.18384935,
       0.15877177, 0.20907979, 0.69504151, 0.70501704, 0.24283856,
       0.17779085, 0.67962854, 0.69443668, 0.17122432, 0.24124814,
       0.15547488, 0.19187937, 0.14312524, 0.68731188, 0.72872838,
       0.16862777, 0.17280251, 0.16088562, 0.74406941, 0.68257574,
       0.256065  , 0.17717418, 0.71965972, 1.10263523, 0.17379287,
       0.22737089, 0.15271783, 0.16817441, 0.17465146, 1.08891541,
       0.18126908, 0.16671547, 0.17944253, 0.69130493, 0.60714922,
       0.70097796, 0.69790387, 0.19056742, 0.19309254, 1.05187745,
       0.68817379, 0.17492435, 0.68053139, 0.19571157, 1.00662

In [40]:

predictions = [round(pred) for pred in predictions]
predictions

[0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,


In [27]:
df_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [10]:
# predictionsの各データを四捨五入してpredictionsに再代入する


### 提出用データの作成
- 提出データは、passenger id, Survivedという2列のデータ形式とします
- データをcsv形式で出力します

In [35]:
# 提出用のDataFrameを作成します
submission = pd.DataFrame({"PassengerId":df_test["PassengerId"], "Survived":predictions})

# 提出用のcsvファイルを作成します
submission.to_csv("submission_7.csv", index=False)

In [38]:
submission

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


### 答え合わせ
- この問題は、世界的なAIコンペサイトであるKaggleの練習問題コンペになっています
- そこでKaggleに上記の結果を提出して、スコア（予測精度）を確認しましょう
- https://www.kaggle.com/competitions/titanic/overview

# ■実践

### 答え合わせの後
- 性別のみの単回帰分析の結果と比較してどうだったでしょうか？
- 年齢は以前の分析で、あまり生死との関係性がないようでした
- ではFareはどうでしょうか？もし、精度向上が認められなかった場合、Fareも生死にはあまり相関がないかもしれません。それをここで確かめてみましょう

In [41]:
#pointbiserialrでFareの相関を確認
from scipy.stats import pointbiserialr

corr_fare,_ = pointbiserialr(df_train["Survived"], df_train["Fare"])
print("Point Biserial Corelation:", corr_fare)

Point Biserial Corelation: 0.25730652238496243


: 

性別のみの単回帰分析で得られた結果は、  
Point Biserial Correlation: 0.5433513806577557なので、  
Fareは性別ほど相関がないようですね